# Data preparation for single-residue variants


## Introduction

<img style="margin-left: 1.5rem" align="right" src="images/data_generation_variants.png" width="400">

This tutorial will demonstrate the use of DeepRank2 for generating single-residue variants (SRVs) graphs and saving them as [HDF5 files](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) files, using [PBD files](<https://en.wikipedia.org/wiki/Protein_Data_Bank_(file_format)>) of protein structures as input.

In this data processing phase, a local neighborhood around the mutated residue is selected for each SRV according to a radius threshold that the user can customize. All atoms or residues within the threshold are mapped as the nodes to a graph and the interactions between them are the edges of the graph. Each node and edge can have several distinct (structural or physico-chemical) features, which are generated and added during the processing phase as well. Optionally, the graphs can be mapped to volumetric grids (i.e., 3D image-like representations), together with their features. Finally, the mapped data are saved as HDF5 files, which can be used for training predictive models (for details see [training_ppi.ipynb](https://github.com/DeepRank/deeprank-core/blob/main/tutorials/training_ppi.ipynb) tutorial). In particular, graphs can be used for the training of Graph Neural Networks (GNNs), and grids can be used for the training of Convolutional Neural Networks (CNNs).


### Input Data

The example data used in this tutorial are available on Zenodo at [this record address](https://zenodo.org/record/8349335). To download the raw data used in this tutorial, please visit the link and download `data_raw.zip`. Unzip it, and save the `data_raw/` folder in the same directory as this notebook. The name and the location of the folder are optional but recommended, as they are the name and the location we will use to refer to the folder throughout the tutorial.

Note that the dataset contains only 96 data points, which is not enough to develop an impactful predictive model, and the scope of its use is indeed only demonstrative and informative for the users.


## Utilities


### Libraries


The libraries needed for this tutorial:


In [ ]:
import glob
import os

import h5py
import matplotlib.image as img
import matplotlib.pyplot as plt
import pandas as pd

from deeprank2.dataset import GraphDataset
from deeprank2.domain.aminoacidlist import amino_acids_by_code
from deeprank2.features import components, contact
from deeprank2.query import QueryCollection, SingleResidueVariantQuery
from deeprank2.utils.grid import GridSettings, MapMethod

### Raw files and paths


The paths for reading raw data and saving the processed ones:


In [ ]:
data_path = os.path.join("data_raw", "srv")
processed_data_path = os.path.join("data_processed", "srv")
os.makedirs(os.path.join(processed_data_path, "residue"))
os.makedirs(os.path.join(processed_data_path, "atomic"))
# Flag limit_data as True if you are running on a machine with limited memory (e.g., Docker container)
limit_data = True

- Raw data are PDB files in `data_raw/srv/pdb/`, which contains atomic coordinates of the protein structure containing the variant.
- Target data, so in our case pathogenic versus benign labels, are in `data_raw/srv/srv_target_values.csv`.
- The final SRV processed data will be saved in `data_processed/srv/` folder, which in turns contains a folder for residue-level data and another one for atomic-level data. More details about such different levels will come a few cells below.


`get_pdb_files_and_target_data` is an helper function used to retrieve the raw pdb files names, SRVs information and target values in a list from the CSV:


In [ ]:
def get_pdb_files_and_target_data(data_path: str) -> tuple[list[str], list[int], list[str], list[str], list[float]]:
    csv_data = pd.read_csv(os.path.join(data_path, "srv_target_values.csv"))
    pdb_files = glob.glob(os.path.join(data_path, "pdb", "*.ent"))
    pdb_files.sort()
    pdb_file_names = [os.path.basename(pdb_file) for pdb_file in pdb_files]
    csv_data_indexed = csv_data.set_index("pdb_file")
    csv_data_indexed = csv_data_indexed.loc[pdb_file_names]
    res_numbers = csv_data_indexed.res_number.tolist()
    res_wildtypes = csv_data_indexed.res_wildtype.tolist()
    res_variants = csv_data_indexed.res_variant.tolist()
    targets = csv_data_indexed.target.tolist()
    pdb_names = csv_data_indexed.index.tolist()
    pdb_files = [data_path + "/pdb/" + pdb_name for pdb_name in pdb_names]

    return pdb_files, res_numbers, res_wildtypes, res_variants, targets


pdb_files, res_numbers, res_wildtypes, res_variants, targets = get_pdb_files_and_target_data(data_path)

if limit_data:
    pdb_files = pdb_files[:15]

## `QueryCollection` and `Query` objects


For each SRV, so for each data point, a query can be created and added to the `QueryCollection` object, to be processed later on. Different types of queries exist, based on the molecular resolution needed:

A query takes as inputs:

- A `.pdb` file, representing the protein structure containing the SRV.
- The resolution (`"residue"` or `"atom"`), i.e. whether each node should represent an amino acid residue or an atom.
- The chain id of the SRV.
- The residue number of the missense mutation.
- The insertion code, used when two residues have the same numbering. The combination of residue numbering and insertion code defines the unique residue.
- The wildtype amino acid.
- The variant amino acid.
- The interaction radius, which determines the threshold distance (in Ångström) for residues/atoms surrounding the mutation that will be included in the graph.
- The target values associated with the query. For each query/data point, in the use case demonstrated in this tutorial will add a 0 if the SRV belongs to the benign class, and 1 if it belongs to the pathogenic one.
- The max edge distance, which is the maximum distance between two nodes to generate an edge between them.
- Optional: The correspondent [Position-Specific Scoring Matrices (PSSMs)](https://en.wikipedia.org/wiki/Position_weight_matrix), per chain identifier, in the form of .pssm files. PSSMs are optional and will not be used in this tutorial.


## Residue-level SRV: `SingleResidueVariantQuery`


In [ ]:
queries = QueryCollection()

influence_radius = 10.0  # radius to select the local neighborhood around the SRV
max_edge_length = 4.5  # ??

print(f"Adding {len(pdb_files)} queries to the query collection ...")
for i in range(len(pdb_files)):
    queries.add(
        SingleResidueVariantQuery(
            pdb_path=pdb_files[i],
            resolution="residue",
            chain_ids="A",
            variant_residue_number=res_numbers[i],
            insertion_code=None,
            wildtype_amino_acid=amino_acids_by_code[res_wildtypes[i]],
            variant_amino_acid=amino_acids_by_code[res_variants[i]],
            targets={"binary": targets[i]},
            influence_radius=influence_radius,
            max_edge_length=max_edge_length,
        ),
    )
    if i + 1 % 20 == 0:
        print(f"{i+1} queries added to the collection.")

print(f"{i+1} queries ready to be processed.\n")

### Notes on `process()` method

Once all queries have been added to the `QueryCollection` instance, they can be processed. Main parameters of the `process()` method, include:

- `prefix` sets the output file location.
- `feature_modules` allows you to choose which feature generating modules you want to use. By default, the basic features contained in `deeprank2.features.components` and `deeprank2.features.contact` are generated. Users can add custom features by creating a new module and placing it in the `deeprank2.feature` subpackage. A complete and detailed list of the pre-implemented features per module and more information about how to add custom features can be found [here](https://deeprank2.readthedocs.io/en/latest/features.html).
  - Note that all features generated by a module will be added if that module was selected, and there is no way to only generate specific features from that module. However, during the training phase shown in `training_ppi.ipynb`, it is possible to select only a subset of available features.
- `cpu_count` can be used to specify how many processes to be run simultaneously, and will coincide with the number of HDF5 files generated. By default it takes all available CPU cores and HDF5 files are squashed into a single file using the `combine_output` setting.
- Optional: If you want to include grids in the HDF5 files, which represent the mapping of the graphs to a volumetric box, you need to define `grid_settings` and `grid_map_method`, as shown in the example below. If they are `None` (default), only graphs are saved.


In [ ]:
grid_settings = GridSettings(  # None if you don't want grids
    # the number of points on the x, y, z edges of the cube
    points_counts=[35, 30, 30],
    # x, y, z sizes of the box in Å
    sizes=[1.0, 1.0, 1.0],
)
grid_map_method = MapMethod.GAUSSIAN  # None if you don't want grids

queries.process(
    prefix=os.path.join(processed_data_path, "residue", "proc"),
    feature_modules=[components, contact],
    cpu_count=8,
    combine_output=False,
    grid_settings=grid_settings,
    grid_map_method=grid_map_method,
)

print(f'The queries processing is done. The generated HDF5 files are in {os.path.join(processed_data_path, "residue")}.')

### Exploring data

As representative example, the following is the HDF5 structure generated by the previous code for `pdb2ooh.ent`, so for one single graph, which represents one protein structure containing a SRV in position 112, for the graph + grid case:

```bash
└── residue-graph:A:112:Threonine->Isoleucine:pdb2ooh
    |
    ├── edge_features
    │   ├── _index
    │   ├── _name
    │   ├── covalent
    │   ├── distance
    │   ├── electrostatic
    │   ├── same_chain
    │   └── vanderwaals
    |
    ├── node_features
    │   ├── _chain_id
    │   ├── _name
    │   ├── _position
    │   ├── diff_charge
    │   ├── diff_hb_donors
    │   ├── diff_hb_acceptors
    │   ├── diff_mass
    │   ├── diff_pI
    │   ├── diff_polarity
    │   ├── diff_size
    │   ├── hb_acceptors
    │   ├── hb_donors
    │   ├── polarity
    │   ├── res_charge
    │   ├── res_mass
    |   ├── res_pI
    |   ├── res_size
    |   ├── res_type
    |   └── variant_res
    |
    ├── grid_points
    │   ├── center
    │   ├── x
    │   ├── y
    │   └── z
    |
    ├── mapped_features
    │   ├── _position_000
    │   ├── _position_001
    │   ├── _position_002
    │   ├── covalent
    │   ├── distance
    │   ├── electrostatic
    │   ├── diff_polarity_000
    │   ├── diff_polarity_001
    │   ├── diff_polarity_002
    │   ├── diff_polarity_003
    |   ├── ...
    |   └── vanderwaals
    |
    └── target_values
        └── binary
```

`edge_features`, `node_features`, `mapped_features` are [HDF5 Groups](https://docs.h5py.org/en/stable/high/group.html) which contain [HDF5 Datasets](https://docs.h5py.org/en/stable/high/dataset.html) (e.g., `_index`, `electrostatic`, etc.), which in turn contains features values in the form of arrays. `edge_features` and `node_features` refer specificly to the graph representation, while `grid_points` and `mapped_features` refer to the grid mapped from the graph. Each data point generated by deeprank2 has the above structure, with the features and the target changing according to the user's settings. Features starting with `_` are present for human inspection of the data, but they are not used for training models.

It is always a good practice to first explore the data, and then make decision about splitting them in training, test and validation sets. There are different possible ways for doing it.


#### Pandas dataframe

The edge and node features just generated can be explored by instantiating the `GraphDataset` object, and then using `hdf5_to_pandas` method which converts node and edge features into a [Pandas](https://pandas.pydata.org/) dataframe. Each row represents a ppi in the form of a graph.


In [ ]:
processed_data = glob.glob(os.path.join(processed_data_path, "residue", "*.hdf5"))
dataset = GraphDataset(processed_data, target="binary")
dataset_df = dataset.hdf5_to_pandas()
dataset_df.head()

We can also generate histograms for looking at the features distributions. An example:


In [ ]:
fname = os.path.join(processed_data_path, "residue", "res_mass_distance_electrostatic")

dataset.save_hist(features=["res_mass", "distance", "electrostatic"], fname=fname)

im = img.imread(fname + ".png")
plt.figure(figsize=(15, 10))
fig = plt.imshow(im)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

#### Other tools

- [HDFView](https://www.hdfgroup.org/downloads/hdfview/), a visual tool written in Java for browsing and editing HDF5 files.
  As representative example, the following is the structure for `pdb2ooh.ent` seen from HDF5View:

  <img style="margin-bottom: 1.5rem" align="centrum" src="images/hdfview_variant.png" width="200">

  Using this tool you can inspect the values of the features visually, for each data point.

- Python packages such as [h5py](https://docs.h5py.org/en/stable/index.html). Examples:


In [ ]:
with h5py.File(processed_data[0], "r") as hdf5:
    # List of all graphs in hdf5, each graph representing
    # a SRV and its sourrouding environment
    ids = list(hdf5.keys())
    print(f"IDs of SRVs in {processed_data[0]}: {ids}")
    node_features = list(hdf5[ids[0]]["node_features"])
    print(f"Node features: {node_features}")
    edge_features = list(hdf5[ids[0]]["edge_features"])
    print(f"Edge features: {edge_features}")
    target_features = list(hdf5[ids[0]]["target_values"])
    print(f"Targets features: {target_features}")
    # Polarity feature for ids[0], numpy.ndarray
    node_feat_polarity = hdf5[ids[0]]["node_features"]["polarity"][:]
    print(f"Polarity feature shape: {node_feat_polarity.shape}")
    # Electrostatic feature for ids[0], numpy.ndarray
    edge_feat_electrostatic = hdf5[ids[0]]["edge_features"]["electrostatic"][:]
    print(f"Electrostatic feature shape: {edge_feat_electrostatic.shape}")

## Atomic-level SRV: `SingleResidueVariantQuery`

Graphs can also be generated at an atomic resolution, very similarly to what has just been done for residue-level.


In [ ]:
queries = QueryCollection()

influence_radius = 10.0  # radius to select the local neighborhood around the SRV
max_edge_length = 4.5  # ??

print(f"Adding {len(pdb_files)} queries to the query collection ...")
for i in range(len(pdb_files)):
    queries.add(
        SingleResidueVariantQuery(
            pdb_path=pdb_files[i],
            resolution="atom",
            chain_ids="A",
            variant_residue_number=res_numbers[i],
            insertion_code=None,
            wildtype_amino_acid=amino_acids_by_code[res_wildtypes[i]],
            variant_amino_acid=amino_acids_by_code[res_variants[i]],
            targets={"binary": targets[i]},
            influence_radius=influence_radius,
            max_edge_length=max_edge_length,
        ),
    )
    if i + 1 % 20 == 0:
        print(f"{i+1} queries added to the collection.")

print(f"{i+1} queries ready to be processed.\n")

In [ ]:
grid_settings = GridSettings(  # None if you don't want grids
    # the number of points on the x, y, z edges of the cube
    points_counts=[35, 30, 30],
    # x, y, z sizes of the box in Å
    sizes=[1.0, 1.0, 1.0],
)
grid_map_method = MapMethod.GAUSSIAN  # None if you don't want grids

queries.process(
    prefix=os.path.join(processed_data_path, "atomic", "proc"),
    feature_modules=[components, contact],
    cpu_count=8,
    combine_output=False,
    grid_settings=grid_settings,
    grid_map_method=grid_map_method,
)

print(f'The queries processing is done. The generated HDF5 files are in {os.path.join(processed_data_path, "atomic")}.')

Again, the data can be inspected using `hdf5_to_pandas` function.


In [ ]:
processed_data = glob.glob(os.path.join(processed_data_path, "atomic", "*.hdf5"))
dataset = GraphDataset(processed_data, target="binary")
dataset_df = dataset.hdf5_to_pandas()
dataset_df.head()

In [ ]:
fname = os.path.join(processed_data_path, "atomic", "atom_charge")
dataset.save_hist(features="atom_charge", fname=fname)

im = img.imread(fname + ".png")
plt.figure(figsize=(8, 8))
fig = plt.imshow(im)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

Note that some of the features are different from the ones generated with the residue-level queries. There are indeed features in `deeprank2.features.components` module which are generated only in atomic graphs, i.e. `atom_type`, `atom_charge`, and `pdb_occupancy`, because they don't make sense only in the atomic graphs' representation.
